                                                * Downloading libraries *                                                                   

In [ ]:
# Note: Running in Python 3.9
!pip3 install nltk rouge-score
!pip3 install torch==2.1.0 torchtext==0.16.0  # Example, use versions compatible with each other
!pip3 install pandas
!pip3 install transformers
!pip3 install datasets
!pip3 install evaluate

In [ ]:
! pip3 install sentencepiece
! pip3 install torchtext
! pip3 install py7zr # for samsum dataset

In [4]:
!pip3 cache purge

Files removed: 401


In [ ]:
import torch
import torchtext
import sentencepiece as spm
print(torch.__version__)
print(torchtext.__version__)

print("SentencePiece version:", spm.__version__)

                                                    Importing libraries                                                                 

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.translate.bleu_score import sentence_bleu
import random
import numpy as np
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
from datasets import load_dataset
import evaluate


                                        Load Models                                                                      

In [16]:
def load_models():

    models = {}
    # Load the pre-trained T5 model and tokenizer
    model_name = "t5-small"  # You can also use "t5-base" or "t5-large" for better performance
    t5_model = T5ForConditionalGeneration.from_pretrained(model_name)
    t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Load pre-trained BART model and tokenizer
    model_name = "facebook/bart-base"
    bart_base_tokenizer = BartTokenizer.from_pretrained(model_name)
    bart_base_model = BartForConditionalGeneration.from_pretrained(model_name)

    model_name = "facebook/bart-large-cnn"
    bart_large_cnn_model = pipeline("summarization", model=model_name)

    models['bart-base'] = [bart_base_model, bart_base_tokenizer]
    models['bart-large-cnn'] = [bart_large_cnn_model]
    models['t5'] = [t5_model, t5_tokenizer]
    return models

models = load_models()

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


                                        Load datasets                                                                      

In [41]:
# Load XSum dataset
xsum_dataset = load_dataset("xsum", split="train[:1%]")
print(xsum_dataset[0]["document"])  # Input text
print("-"*50)
print(xsum_dataset[0]["summary"])   # expected summary

print("="*100)
# Load CNN/DailyMail dataset
cnn_dailymail_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")
print(cnn_dailymail_dataset["article"][0])  # Input text
print("-"*50)
print(cnn_dailymail_dataset["highlights"][0])  # expected summary


The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is al

                                        Test and Evaluate Models                                                                      

In [63]:

# Function to summarize text using T5
def summarize_text(text, model_name, max_length=130, min_length=30):
    max_input_length = 1024  # For BART and most Transformer models
    if model_name == 'bart-large-cnn':
        model = models['bart-large-cnn'][0]
        text = text[:max_input_length]  # Cut off text to avoid exceed input limitation of the model
        summary = model(text, max_length, min_length, do_sample=False)
        return summary[0]['summary_text']
    

    elif model_name == 't5': 
        model, tokenizer = models['t5']
    elif model_name == 'bart-base': 
        model, tokenizer = models['bart-base']
    else:
        raise ValueError(f"Model {model_name} not recognized.")
    # Prepare the text for summarization
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    
    # Generate summary ids
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length = max_length,
        min_length = min_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test and evaluate models
def evaluate_models(xsum_dataset, cnn_dailymail_dataset):
    rouge = evaluate.load("rouge")
    results = []
    # The models being tested
    model_names = ['t5', 'bart-base', 'bart-large-cnn']

    # Go through datasets and models 
    for dataset_name, dataset, text_key, summary_key in [
        ("Xsum Dataset", xsum_dataset, "document", "summary"),
        ("CNN/DailyMail Dataset", cnn_dailymail_dataset, "article", "highlights")
    ]:
        print(f"\nEvaluating on {dataset_name}\n")
        for idx in range(3):  # Choose top 3 data to test
            text = dataset[idx][text_key]
            expected_summary = dataset[idx][summary_key]

            print(f"Original Text:\n{text}\n")
            print(f"Expected Summary:\n{expected_summary}\n")

            # Go through models
            for model in model_names:
                predicted_summary = summarize_text(text, model)
                
                # Print the original text, expected summary, and predicted summary
                print(f"Model: {model}\n")
                print(f"Predicted Summary:\n{predicted_summary}\n")
                print('-'*50)

                # Calculate ROUGE 
                rouge_result = rouge.compute(predictions=[predicted_summary], references=[expected_summary])

                # Save results to the table
                results.append({
                    "Model": model,
                    "Dataset": dataset_name,
                    "ROUGE-1": rouge_result['rouge1'],
                    "ROUGE-2": rouge_result['rouge2'],
                    "ROUGE-L": rouge_result['rougeL']
                })
            print("="*100)
    # Convert results to dataFrame
    result_df = pd.DataFrame(results)
    return result_df
    

In [64]:
result_df = evaluate_models(xsum_dataset, cnn_dailymail_dataset)


Evaluating on Xsum Dataset

Original Text:
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - 

Ignoring args : (130, 30)


Model: bart-base

Predicted Summary:
summarize: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Credit:Scottish Borders CouncilRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.Image copyright Getty Images Image caption Many businesses and householders were affected by floodingIn Newton Stewart after the River Cree overflowed into the town.The River Cree has since receded.First Minister Nicola Sturgeon visited the area to inspect the damage.The waters breached

--------------------------------------------------
Model: bart-large-cnn

Predicted Summary:
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Many roads in Peeblesshire remain badly affected by standing water. The waters breached a retaining wall, flooding many commercial properties on

Ignoring args : (130, 30)


Model: bart-base

Predicted Summary:
summarize: A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned

--------------------------------------------------
Model: bart-large-cnn

Predicted Summary:
Fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST. One of the tour groups is from Germany, the other from China and Taiwan. The driver of one of the buses said many of the passengers had left personal belongings on board and these had bee

...

Model: bart-base

Predicted Summary:
summarize: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy

--------------------------------------------------
Model: bart-large-cnn

Predicted Summary:
Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe says he has no plans to fritter his cash away on fast cars, drink and parties. At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see "Hostel: Part II"

--------------

Ignoring args : (130, 30)


Model: bart-base

Predicted Summary:
summarize: Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes you inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "Forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often

--------------------------------------------------
Model: bart-large-cnn

Predicted Summary:
Miami-Dade pretrial detention facility houses mentally ill inmates. Most often, they face drug charges or charges of assaulting an officer. Judge: Arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arr

Ignoring args : (130, 30)


Model: bart-base

Predicted Summary:
summarize: MINNEAPOLIS, Minnesota (CNN) -- Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. "The whole bridge from one side of the Mississippi to the other just completely gave way, fell all the way down," survivor Gary Babineau told CNN. "I probably had a 30-, 35-foot free fall. And there're cars in the water, there's cars on fire. The whole bridge is down." He said his back was injured but he determined he could move around. "We realized there was a school bus right next to me, and me and a

--------------------------------------------------
Model: bart-large-cnn

Predicted Summary:
"The whole bridge from one side of the Mississippi to the other just completely gave way" "I probably had a 30-, 35-foot free fall," survivor Gary Babineau says. "There's cars in the water, there's cars on fire. The whole bridge is down," he says.

--------------------------------------------------


In [65]:
result_df

,Model,Dataset,ROUGE-1,ROUGE-2,ROUGE-L
0,t5,Xsum Dataset,0.117647,0.000000,0.078431
1,bart-base,Xsum Dataset,0.119658,0.017391,0.102564
2,bart-large-cnn,Xsum Dataset,0.080000,0.000000,0.053333
3,t5,Xsum Dataset,0.280000,0.000000,0.160000
4,bart-base,Xsum Dataset,0.153846,0.015625,0.076923
5,bart-large-cnn,Xsum Dataset,0.142857,0.029412,0.085714
6,t5,Xsum Dataset,0.092308,0.031746,0.061538
7,bart-base,Xsum Dataset,0.180328,0.050000,0.098361
8,bart-large-cnn,Xsum Dataset,0.242424,0.062500,0.121212
9,t5,CNN/DailyMail Dataset,0.288660,0.231579,0.268041


In [66]:
# Group by model and dataset, and calculate the mean ROUGE value of each group
grouped_df = result_df.groupby(['Model', 'Dataset']).mean().reset_index()
grouped_df

,Model,Dataset,ROUGE-1,ROUGE-2,ROUGE-L
0,bart-base,CNN/DailyMail Dataset,0.324020,0.172484,0.226412
1,bart-base,Xsum Dataset,0.151277,0.027672,0.092616
2,bart-large-cnn,CNN/DailyMail Dataset,0.354129,0.179741,0.267694
3,bart-large-cnn,Xsum Dataset,0.155094,0.030637,0.086753
4,t5,CNN/DailyMail Dataset,0.255494,0.131615,0.208819
5,t5,Xsum Dataset,0.163318,0.010582,0.099990
